# Lesson 3 - Agent with data access

In this lesson you will learn how to create an agent with access to Financial data and convert currency values.

## Get Auth your Auth credentials

## Connect to your Weaviate instance

In [ ]:
from weaviate_connect import WeaviateConnect
client = WeaviateConnect.connect()

In [ ]:
client.collections.list_all()

## Quickly test a vector query

In [ ]:
collection = client.collections.use("FinancialArticles")
res = collection.query.near_text(
    query="Tech stock",
    target_vector="body",
    auto_limit=1,
)

for item in res.objects:
    print(item.properties)

## Build Financial Search tool

In [ ]:
from weaviateconnect import WeaviateConnect

from typing import Annotated, List
from pydantic import Field


def search_financial_articles_tool(
        query: Annotated[str, Field(description="Search query term")]
    ) -> Annotated[List, Field(description="A list of database object. See .properties for the data items.")]:
    """
    A tool to search through Financial Articles.
    """

    print(f"🤖 TOOL: search_financial_articles_tool({query})")

    # Get the connect to Weaviate first
    client = WeaviateConnect.connect()
    
    # Run the query
    articles = client.collections.use("FinancialArticles")
    response = articles.query.near_text(
        query,
        target_vector="title",
        limit=5,
    )
    
    return response.objects

## Other tools

In [ ]:
!pip install requests

In [ ]:
import requests

def refusal_tool() -> Annotated[str, Field(description="The response provides you with instructions on how to respond to the user")]:
    """This is a refusal tool, it tells you how to respond to the user"""
    print("🤖 TOOL: refusal_tool()")

    return """
        Make sure to use the nickname_tool to respond.
        Appologise that you can't help them.
        List the names of the tools that you have available, but don't mention refusal_tool and nickname_tool
        And tell them to talk to Sebastian to upgrade you."""

def currency_exchange_tool(
        currency_from: Annotated[str, Field(description="Source currency code (e.g., 'usd', 'eur')")],
        currency_to: Annotated[str, Field(description="Target currency code (e.g., 'usd', 'eur', 'btc')")]
    ) -> Annotated[float, Field(description="Exchange rate from source to target currency, or 1.0 if not found")]:
    """
    Get the exchange rate between two currencies.
    """

    print(f"🤖 TOOL: Checking currency rates {currency_from}->{currency_to}")

    # Convert to lowercase to match API format
    currency_from = currency_from.lower()
    currency_to = currency_to.lower()

    # If same currency, return 1
    if currency_from == currency_to:
        return 1.0

    try:
        # Make API call to get exchange rates for the source currency
        url = f"https://latest.currency-api.pages.dev/v1/currencies/{currency_from}.json"
        response = requests.get(url)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Warning: Unable to fetch exchange rates for '{currency_from}'. HTTP status: {response.status_code}")
            return 1.0
        
        # Parse the JSON response
        data = response.json()
        
        # Check if the source currency data exists
        if currency_from not in data:
            print(f"Warning: Source currency '{currency_from}' not found in the response.")
            return 1.0
        
        # Get the exchange rates for the source currency
        exchange_rates = data[currency_from]
        
        # Check if the target currency is available
        if currency_to not in exchange_rates:
            print(f"Warning: Target currency '{currency_to}' not found for source currency '{currency_from}'.")
            return 1.0
        
        # Return the exchange rate
        print(f"The exchange rate for {currency_from} to {currency_to} is {exchange_rates[currency_to]}")
        return float(exchange_rates[currency_to])
    
    except Exception as e:
        print(f"Warning: An unexpected error occurred: {e}")
        return 1.0

## Create an agent

In [ ]:
from pydantic_ai import Agent
from pydantic_ai.tools import Tool

def prepare_agent() -> Agent:
    return Agent(
        # model="bedrock:anthropic.claude-3-5-sonnet-20241022-v2:0",
        model="openai:gpt-4o",
        instructions="""
        You are a helpful assistant answering financial questions.
        You have access to the following tools:
        - search_financial_articles_tool
        - currency_exchange_tool

        You should only try to provide the response only with the provided tools.
        Use refusal_tool to understand how to respond when you don't have the tools to answer.
        """,
        tools=[
            Tool(search_financial_articles_tool, takes_ctx=False),
            Tool(currency_exchange_tool, takes_ctx=False),
            Tool(refusal_tool, takes_ctx=False),
        ]
    )

agent = prepare_agent()

## Test the agent

In [ ]:
result = await agent.run("What are popular tech stocks? Which one grew the most?")
print(result.output)

## Run the agent in a loop

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

COL_BLUE = '\033[94m'
COL_RESET = '\033[0m'

async def main():
    logger.info("Starting Pydantic AI demo with Weaviate search and text processing tools")

    print("Welcome to the Pydantic AI Demo!")
    print("You can ask me to:")
    print("- Search for documents (e.g., 'Search for machine learning papers')")
    print("- Count characters in text (e.g., 'Count the letter r in programming')")
    print("Type 'quit' to exit.\n")

    agent = prepare_agent()
    result = None

    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ['quit', 'exit']:
                break

            if user_input.lower() in ['clear', 'clr']:
                result = None
                continue

            if(result == None):
                result = await agent.run(user_input)
            else:
                result = await agent.run(user_input, message_history=result.all_messages())

            print(f"{COL_BLUE}Assistant: {result.output}\n{COL_RESET}")

        except KeyboardInterrupt:
            break
        except Exception as e:
            logger.error(f"Error processing request: {e}")
            print(f"Sorry, there was an error: {e}\n")

    print("Goodbye!")

await main()
